In [2]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from geopy.geocoders import Nominatim
from collections import Counter
import time

# Sample job data, assuming 'innerText' field from the dictionary,
job_data = [
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President & Chief Executive Officer\nPresident & Chief Executive Officer\nInkinen Executive Search\nWaimea, HI\nActively Hiring\n2 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President/CEO\nPresident/CEO\nSalt Creek Capital\nGreater St. Louis\nActively Hiring\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer\nChief Executive Officer\nAC Future\nIrvine, CA\nMedical insurance +3\xa0benefits\n1 month ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President/CEO\nPresident/CEO\nSalt Creek Capital\nSan Francisco Bay Area\nActively Hiring\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President and Chief Executive Officer\nPresident and Chief Executive Officer\nColorado Oil & Gas Association\nDenver, CO\n5 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President/CEO\nPresident/CEO\nSalt Creek Capital\nBoston, MA\nActively Hiring\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive\nChief Executive\nAugment Jobs\nUnited States\n3 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer, Huddle\nChief Executive Officer, Huddle\nPalm Venture Studios\nAustin, TX\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer\nChief Executive Officer\nGemTek\nBeverly Hills, CA\n6 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President/CEO\nPresident/CEO\nSalt Creek Capital\nLas Vegas, NV\nActively Hiring\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President/CEO\nPresident/CEO\nSalt Creek Capital\nRichmond, VA\nActively Hiring\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President/CEO\nPresident/CEO\nSalt Creek Capital\nAtlanta Metropolitan Area\nActively Hiring\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President & Chief Executive Officer\nPresident & Chief Executive Officer\nTennessee Chamber of Commerce & Industry\nNashville, TN\n2 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO and Co-Founder\nCEO and Co-Founder\nPangaea Global Associates\nArkansas, United States\nMedical insurance +4\xa0benefits\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President and Chief Executive Officer\nPresident and Chief Executive Officer\nBay Cove Human Services\nBoston, MA\nActively Hiring +7\xa0benefits\n6 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President and Chief Executive Officer\nPresident and Chief Executive Officer\nThe Jacobson Group\nMichigan, United States\nActively Hiring\n2 months ago', 'attr,ibutes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer\nChief Executive Officer\nKMA Human Resources Consulting\nAugusta, ME\nBe an early applicant +4\xa0benefits\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President and CEO\nPresident and CEO\nBrewers Association\nBoulder, CO\n4 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer (CEO)\nChief Executive Officer (CEO)\nVeracity Software Inc\nVancouver, WA\nBe an early applicant\n1 month ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer\nChief Executive Officer\nEncompass Health\nPlano, TX\nActively Hiring +4\xa0benefits\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President and Chief Executive Officer\nPresident and Chief Executive Officer\nThe Jacobson Group\nMinnesota, United States\nActively Hiring\n3 months ago', 'att,ributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President / Chief Executive Officer\nPresident / Chief Executive Officer\nThe Moran Company Executive Search\nClearwater, FL\n2 months ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer\nChief Executive Officer\nPonca City Development Authority\nPonca City, OK\nBe an early applicant\n3 months ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO & Co-Founder - WealthTech\nCEO & Co-Founder - WealthTech\nFutureSight\nUnited States\nBe an early applicant\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer\nChief Executive Officer\nVerge\nHouston, TX\nActively Hiring +4\xa0benefits\n2 months ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nSan Jose, CA\nBe an early applicant\n3 weeks ag,o', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO\nCEO\nAugment Jobs\nUnited States\n4 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President\nPresident\nPhifer & Company\nNew York, NY\nActively Hiring\n6 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President\nPresident\nTransport ATL\nAnchorage, AK\nBe an early applicant\n1 day ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President & Chief Executive Officer (CEO)\nPresident & Chief Executive Officer (CEO)\nThe Moran Company Executive Search\nOmaha, NE\n1 month ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President\nPresident\nConfidential\nMahwah, NJ\nActively Hiring +4\xa0benefits\n6 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nPalo Alto, CA\nBe an early applicant\n3 weeks a,go', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nSan Francisco, CA\nBe an early applicant\n3 wee,ks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer - THC (Healthcare)\nChief Executive Officer - THC (Healthcare)\nLeap Brands\nWinter Park, FL\n5 months ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nLos Angeles, CA\nBe an early applicant\n3 weeks, ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President\nPresident\nExecutive Leadership Solutions\nDenver, CO\nActively Hiring +6\xa0benefits\n2 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nBoulder, CO\nBe an early applicant\n3 weeks ago,', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President and Chief Executive Officer\nPresident and Chief Executive Officer\nThe Jacobson Group\nMaryland, United States\nActively Hiring\n2 months ago', 'attr,ibutes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nMiami, FL\nBe an early applicant\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nSeattle, WA\nBe an early applicant\n3 weeks ago,', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President\nPresident\nBlue Signal Search\nUnited States\nActively Hiring\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO-In-Training, Executive Director\nCEO-In-Training, Executive Director\nPennant\nGreen Bay, WI\nActively Hiring\n2 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Entrepreneur in Residence (Future CEO / Founder)\nEntrepreneur in Residence (Future CEO / Founder)\nFutureSight\nDenver, CO\nBe an early applicant\n3 weeks ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President & Chief Executive Officer (CEO)\nPresident & Chief Executive Officer (CEO)\nConnexus® Energy\nRamsey, MN\n3 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer / President\nChief Executive Officer / President\nGet It Recruit - Executive\nClearwater, FL\nBe an early applicant\n6 days ago', 'attri,butes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO - Manufacturing #1510\nCEO - Manufacturing #1510\nKeller Executive Search International\nCleveland, OH\n1 month ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer: (Transportation Industry)\nChief Executive Officer: (Transportation Industry)\nEast 57th Street Partners\nUnited States\nActively Hirin,g\n8 hours ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President\nPresident\nEmployment Resource Group, Inc.\nHudson, WI\nActively Hiring\n6 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Founder\nFounder\nAugment Jobs\nUnited States\nBe an early applicant\n5 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer, Fitz Custom\nChief Executive Officer, Fitz Custom\nPalm Venture Studios\nUnited States\n3 months ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO - Manufacturing #1510\nCEO - Manufacturing #1510\nKeller Executive Search International\nAkron, OH\n1 month ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President USA&Canada\nPresident USA&Canada\nKienbaum\nGreater Chicago Area\nActively Hiring +4\xa0benefits\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Founder\nFounder\nAugment Jobs\nUnited States\nBe an early applicant\n6 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Founder\nFounder\nAugment Jobs\nUnited States\nBe an early applicant\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'President & CEO\nPresident & CEO\nVisit Oxnard\nOxnard, CA\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO - Manufacturing #1510\nCEO - Manufacturing #1510\nKeller Executive Search International\nMedina, OH\nBe an early applicant\n1 month ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer - Construction\nChief Executive Officer - Construction\nLeap Brands\nThonotosassa, FL\n3 months ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Founder\nFounder\nAugment Jobs\nUnited States\nBe an early applicant\n1 week ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'Chief Executive Officer\nChief Executive Officer\nMaryland Nonprofits\nEast Providence, RI\nBe an early applicant\n4 days ago', 'attributes': {}},
{'tagName': 'LI', 'className': 'N/A', 'id': 'N/A', 'innerText': 'CEO - Manufacturing #1510\nCEO - Manufacturing #1510\nKeller Executive Search International\nOrrville, OH\n1 month ago', 'attributes': {}},
]

# Extract cities and states,
locations = []
for job in job_data:
    lines = job['innerText'].split('\n')
    if len(lines) >= 3:
        locations.append(lines[2])

# Count occurrences
location_counts = Counter(locations)

# Convert to DataFrame
df = pd.DataFrame(location_counts.items(), columns=['Location', 'Job Count'])

# Geocode locations
geolocator = Nominatim(user_agent="geoapi")
latitudes = []
longitudes = []

for location in df['Location']:
    try:
        loc = geolocator.geocode(location)
        if loc:
            latitudes.append(loc.latitude)
            longitudes.append(loc.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
        time.sleep(1)  # Pause to respect rate limits
    except Exception as e:
        latitudes.append(None)
        longitudes.append(None)

df['Latitude'] = latitudes
df['Longitude'] = longitudes
df.dropna(inplace=True)

# Plot heat map
fig = px.density_mapbox(df, lat='Latitude', lon='Longitude', z='Job Count',
                        radius=20, center=dict(lat=37.0902, lon=-95.7129), zoom=3,
                        mapbox_style="carto-positron",
                        title="Heatmap of Job Locations for Specified Title in the U.S.")

fig.update_layout(title_x=0.5)
fig.show()


```### Explanation:

1. **Data Extraction**: This pulls city and state from each job entry.
2. **Geocoding**: Converts cities to geographic coordinates.
3. **Data Cleaning**: Removes rows without valid coordinates.
4. **Plotting**: Uses Plotly’s `density_mapbox` to overlay a heat map on the U.S., with higher density showing more jobs.

This visualization can be embedded into your portfolio site using Plotly’s HTML export capabilities. You can expand this further by adding filters for job titles or company names, and even layering in job trends over time.

In [ ]:
To create a heat map showing job locations on a U.S. map, you can follow these steps to extract location data and use Python libraries like `pandas`, `geopandas`, and `plotly`:

1. **Extract City and State Information**: Parse the location data for each job posting to extract cities and states.
  
2. **Geocode Locations**: Use a library like `geopy` to convert city/state names into latitude and longitude coordinates.

3. **Count Job Occurrences by Location**: Count the occurrences of each city in the job data to determine job density per location.

4. **Visualize with Plotly**: Create a heat map overlaying these points on a U.S. map. Here’s an example code outline:

```python



In [ ]:
import React, { useState } from 'react';

const JobHeatmap = () => {
  const jobsByState = {
    'AK': 1, 
    'CA': 5, 
    'CO': 4, 
    'FL': 4, 
    'GA': 1, 
    'HI': 1, 
    'IL': 1, 
    'MA': 2, 
    'ME': 1, 
    'MD': 1, 
    'MN': 2, 
    'NJ': 1, 
    'NV': 1, 
    'NY': 1, 
    'OH': 4, 
    'OK': 1, 
    'RI': 1, 
    'TX': 2, 
    'VA': 1, 
    'WA': 2, 
    'WI': 2
  };

  const [hoveredState, setHoveredState] = useState(null);

  const getColor = (count) => {
    const maxCount = 5; // Maximum jobs in a state
    const intensity = count ? (count / maxCount) : 0;
    return `rgb(0, 0, ${Math.floor(255 * (0.3 + 0.7 * intensity))})`;
  };

  return (
    <div className="w-full max-w-4xl mx-auto p-4 bg-white rounded-lg shadow">
      <h2 className="text-2xl font-bold mb-4 text-center">Executive Job Distribution Across the US</h2>
      
      <div className="relative">
        <div className="mb-4 text-center">
          {hoveredState && (
            <div className="text-lg">
              {hoveredState}: {jobsByState[hoveredState] || 0} jobs
            </div>
          )}
        </div>

        <svg viewBox="0 0 1000 600" className="w-full h-auto">
          <rect x="0" y="0" width="1000" height="600" fill="#f0f0f0" />
          
          {/* Sample state rectangles - simplified representation */}
          {Object.entries(jobsByState).map(([state, count]) => {
            // Simplified positioning - just for demonstration
            const positions = {
              'CA': { x: 100, y: 200, width: 50, height: 80 },
              'TX': { x: 300, y: 300, width: 70, height: 60 },
              'FL': { x: 650, y: 400, width: 50, height: 30 },
              'NY': { x: 700, y: 150, width: 30, height: 30 },
              // Add more state positions as needed
            };

            const statePos = positions[state];
            if (!statePos) return null;

            return (
              <g key={state}>
                <rect
                  x={statePos.x}
                  y={statePos.y}
                  width={statePos.width}
                  height={statePos.height}
                  fill={getColor(count)}
                  stroke="#fff"
                  strokeWidth="1"
                  onMouseEnter={() => setHoveredState(state)}
                  onMouseLeave={() => setHoveredState(null)}
                  className="cursor-pointer hover:opacity-80"
                />
                <text
                  x={statePos.x + statePos.width/2}
                  y={statePos.y + statePos.height/2}
                  textAnchor="middle"
                  fill="#ffffff"
                  fontSize="12"
                >
                  {state}
                </text>
              </g>
            );
          })}
        </svg>
      </div>

      <div className="mt-4">
        <div className="flex justify-between items-center px-4">
          <span>Fewer Jobs</span>
          <div className="flex gap-1">
            {[0, 1, 2, 3, 4, 5].map((level) => (
              <div
                key={level}
                className="w-8 h-4"
                style={{ backgroundColor: getColor(level) }}
              />
            ))}
          </div>
          <span>More Jobs</span>
        </div>
      </div>
    </div>
  );
};

export default JobHeatmap;

In [ ]:
import React, { useState } from 'react';

const JobHeatmap = () => {
  const jobsByState = {
    'AK': 1, 
    'CA': 5, 
    'CO': 4, 
    'FL': 4, 
    'GA': 1, 
    'HI': 1, 
    'IL': 1, 
    'MA': 2, 
    'ME': 1, 
    'MD': 1, 
    'MN': 2, 
    'NJ': 1, 
    'NV': 1, 
    'NY': 1, 
    'OH': 4, 
    'OK': 1, 
    'RI': 1, 
    'TX': 2, 
    'VA': 1, 
    'WA': 2, 
    'WI': 2
  };

  const [hoveredState, setHoveredState] = useState(null);

  const getColor = (count) => {
    const maxCount = 5; // Maximum jobs in a state
    const intensity = count ? (count / maxCount) : 0;
    return `rgb(0, 0, ${Math.floor(255 * (0.3 + 0.7 * intensity))})`;
  };

  return (
    <div className="w-full max-w-4xl mx-auto p-4 bg-white rounded-lg shadow">
      <h2 className="text-2xl font-bold mb-4 text-center">Executive Job Distribution Across the US</h2>
      
      <div className="relative">
        <div className="mb-4 text-center">
          {hoveredState && (
            <div className="text-lg">
              {hoveredState}: {jobsByState[hoveredState] || 0} jobs
            </div>
          )}
        </div>

        <svg viewBox="0 0 1000 600" className="w-full h-auto">
          <rect x="0" y="0" width="1000" height="600" fill="#f0f0f0" />
          
          {/* Sample state rectangles - simplified representation */}
          {Object.entries(jobsByState).map(([state, count]) => {
            // Simplified positioning - just for demonstration
            const positions = {
              'CA': { x: 100, y: 200, width: 50, height: 80 },
              'TX': { x: 300, y: 300, width: 70, height: 60 },
              'FL': { x: 650, y: 400, width: 50, height: 30 },
              'NY': { x: 700, y: 150, width: 30, height: 30 },
              // Add more state positions as needed
            };

            const statePos = positions[state];
            if (!statePos) return null;

            return (
              <g key={state}>
                <rect
                  x={statePos.x}
                  y={statePos.y}
                  width={statePos.width}
                  height={statePos.height}
                  fill={getColor(count)}
                  stroke="#fff"
                  strokeWidth="1"
                  onMouseEnter={() => setHoveredState(state)}
                  onMouseLeave={() => setHoveredState(null)}
                  className="cursor-pointer hover:opacity-80"
                />
                <text
                  x={statePos.x + statePos.width/2}
                  y={statePos.y + statePos.height/2}
                  textAnchor="middle"
                  fill="#ffffff"
                  fontSize="12"
                >
                  {state}
                </text>
              </g>
            );
          })}
        </svg>
      </div>

      <div className="mt-4">
        <div className="flex justify-between items-center px-4">
          <span>Fewer Jobs</span>
          <div className="flex gap-1">
            {[0, 1, 2, 3, 4, 5].map((level) => (
              <div
                key={level}
                className="w-8 h-4"
                style={{ backgroundColor: getColor(level) }}
              />
            ))}
          </div>
          <span>More Jobs</span>
        </div>
      </div>
    </div>
  );
};

export default JobHeatmap;